In [ ]:
!pip -q install ultralytics adversarial-robustness-toolbox torch torchvision

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 42.3 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import glob
import shutil
import random
import time
import yaml

import numpy as np
import cv2
from tqdm import tqdm

import torch
from ultralytics import YOLO

# ART imports
from art.estimators.object_detection.object_detector import ObjectDetector
from art.attacks.evasion import RobustDPatch

# ---------- PATHS & CONFIG ----------
MODEL_PATH      = "/content/drive/MyDrive/MN-20-Credit/models/yolov9/best.pt"
VAL_IMAGES_DIR  = "/content/drive/MyDrive/MN-20-Credit/dota-yolo/images/val"
VAL_LABELS_DIR  = "/content/drive/MyDrive/MN-20-Credit/dota-yolo/labels/val"

ATTACK_ROOT     = "/content/drive/MyDrive/MN-20-Credit/dota-yolo/DPatch_Attack"
ATTACK_IMG_DIR  = os.path.join(ATTACK_ROOT, "images/val")
ATTACK_LBL_DIR  = os.path.join(ATTACK_ROOT, "labels/val")

NAMES = [
    "plane","ship","storage tank","baseball diamond","tennis court",
    "basketball court","ground track field","harbor","bridge","large vehicle",
    "small vehicle","helicopter","roundabout","soccer ball field","swimming pool",
    "container crane"
]

# single canonical size
IMG_SIZE   = 640
CONF_THRES = 0.001
DEVICE     = "cuda" if torch.cuda.is_available() else "cpu"   # used for estimator and yolo.val

# patch hyperparams (tune as needed)
PATCH_SIZE    = 128
PATCH_ITERS   = 400
BATCH_SIZE    = 1
LEARNING_RATE = 2.5
SEED = 42

# ---------- utilities ----------
def ensure_clean_dir(path: str):
    if os.path.isdir(path):
        shutil.rmtree(path)
    os.makedirs(path, exist_ok=True)

def load_yolo_label_as_art_boxes(lbl_path, img_w, img_h):
    """
    YOLO txt (class xc yc w h normalized) -> dict('boxes','labels') with xyxy (pixels)
    """
    if not os.path.isfile(lbl_path):
        return {'boxes': np.zeros((0,4), dtype=np.float32),
                'labels': np.zeros((0,), dtype=np.int64)}
    boxes, labels = [], []
    with open(lbl_path, 'r') as f:
        for ln in f:
            parts = ln.strip().split()
            if len(parts) < 5:
                continue
            try:
                cls = int(parts[0])
                xc, yc, w, h = map(float, parts[1:5])
            except Exception:
                continue
            x1 = (xc - w/2.0) * img_w
            y1 = (yc - h/2.0) * img_h
            x2 = (xc + w/2.0) * img_w
            y2 = (yc + h/2.0) * img_h
            # clip per-axis correctly
            x1 = float(np.clip(x1, 0.0, float(img_w)))
            x2 = float(np.clip(x2, 0.0, float(img_w)))
            y1 = float(np.clip(y1, 0.0, float(img_h)))
            y2 = float(np.clip(y2, 0.0, float(img_h)))
            if x2 <= x1 or y2 <= y1:
                continue
            boxes.append([x1, y1, x2, y2])
            labels.append(cls)
    if not boxes:
        return {'boxes': np.zeros((0,4), dtype=np.float32),
                'labels': np.zeros((0,), dtype=np.int64)}
    return {'boxes': np.asarray(boxes, np.float32),
            'labels': np.asarray(labels, np.int64)}

def _mean_objectness_from_raw(raw):
    """
    Accepts raw outputs from YOLO head (pre-NMS).
    Handles list/tuple (multi-scale) or a single tensor.
    Uses [...,4] as objectness channel.
    """
    if isinstance(raw, (list, tuple)):
        vals = []
        for r in raw:
            vals.append(r[..., 4].mean())
        return torch.stack(vals).mean()
    return raw[..., 4].mean()

# ---------- ART estimator for Ultralytics YOLO ----------
class YoloV9ARTDetector(ObjectDetector):
    def __init__(self, yolo_obj: YOLO, input_shape=(3, IMG_SIZE, IMG_SIZE),
                 clip_values=(0.0, 255.0), channels_first=True, device='cpu'):
        self.yolo = yolo_obj
        self.pt_model = getattr(yolo_obj, "model", None)
        if self.pt_model is None:
            raise RuntimeError("Could not access underlying torch model via yolo.model")
        self._input_shape = input_shape
        self._channels_first = channels_first
        self.device = torch.device(device)

        # ART BaseEstimator init (support multiple ART versions)
        try:
            super().__init__(self.pt_model, clip_values=clip_values)
        except TypeError:
            super().__init__(model=self.pt_model, clip_values=clip_values)

        # move model to device and freeze params
        self.pt_model.to(self.device).eval()
        for p in self.pt_model.parameters():
            p.requires_grad = False

    @property
    def input_shape(self):
        return self._input_shape

    @property
    def channels_first(self):
        return self._channels_first

    @property
    def native_label_is_pytorch_format(self) -> bool:
        return False

    def fit(self, x, y, **kwargs):
        return

    def predict(self, x, **kwargs):
        arr = x
        if isinstance(arr, torch.Tensor):
            arr = arr.detach().cpu().numpy()
        # to NHWC uint8 for ultralytics.predict
        if arr.ndim == 4 and self.channels_first:
            arr = np.transpose(arr, (0, 2, 3, 1))
        if arr.dtype != np.uint8:
            if arr.max() <= 1.0:
                arr = (arr * 255.0).round().astype(np.uint8)
            else:
                arr = np.clip(arr, 0, 255).astype(np.uint8)

        results = self.yolo.predict(source=[im for im in arr],
                                    imgsz=self._input_shape[-1],
                                    batch=len(arr),
                                    verbose=False)
        outs = []
        for res in results:
            if hasattr(res, "boxes") and len(res.boxes):
                boxes = res.boxes.xyxy.cpu().numpy().astype(np.float32)
                scores = res.boxes.conf.cpu().numpy().astype(np.float32)
                labels = res.boxes.cls.cpu().numpy().astype(np.int64)
            else:
                boxes = np.zeros((0,4), np.float32)
                scores = np.zeros((0,), np.float32)
                labels = np.zeros((0,), np.int64)
            outs.append({"boxes": boxes, "labels": labels, "scores": scores})
        return outs

    def loss_gradient(self, x, y=None, **kwargs):
        # to tensor, channels-first
        if isinstance(x, np.ndarray):
            t = torch.from_numpy(x)
        else:
            t = x.clone().detach()

        if not self.channels_first:
            t = t.permute(0, 3, 1, 2)  # NHWC->NCHW

        t = t.float()
        if t.max() <= 1.0:
            t = t * 255.0
        t = t.to(self.device)

        # adapt dtype to model if model in fp16
        model_dtype = next(self.pt_model.parameters()).dtype
        if model_dtype == torch.float16:
            t = t.half()

        t.requires_grad_(True)
        prev_training = self.pt_model.training
        try:
            self.pt_model.train()
            torch.set_grad_enabled(True)

            raw = self.pt_model(t)               # differentiable forward (pre-NMS if training forward supported)
            mean_obj = _mean_objectness_from_raw(raw)
            loss = -mean_obj                     # untargeted: suppress objectness

            self.pt_model.zero_grad(set_to_none=True)
            if t.grad is not None:
                t.grad.zero_()
            loss.backward()

            grad = t.grad.detach().cpu().numpy().astype(np.float32)
            if not self.channels_first:
                grad = np.transpose(grad, (0, 2, 3, 1))
            return grad
        finally:
            if prev_training:
                self.pt_model.train()
            else:
                self.pt_model.eval()

# ---------- seed ----------
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

print(f"DEVICE={DEVICE}, IMG_SIZE={IMG_SIZE}")

# ---------- load model & build estimator ----------
print("Loading Ultralytics model...")
yolo = YOLO(MODEL_PATH)

est = YoloV9ARTDetector(
    yolo_obj=yolo,
    input_shape=(3, IMG_SIZE, IMG_SIZE),
    clip_values=(0.0, 255.0),
    channels_first=True,
    device=DEVICE
)
print("ART estimator created.")

# ---------- prepare training subset from val ----------
exts = (".jpg", ".jpeg", ".png", ".bmp", ".tif", ".tiff")
all_val_imgs = [p for p in sorted(glob.glob(os.path.join(VAL_IMAGES_DIR, "*"))) if p.lower().endswith(exts)]
if len(all_val_imgs) == 0:
    raise RuntimeError("No validation images found. Check VAL_IMAGES_DIR and paths.")

N_TRAIN = min(32, max(12, len(all_val_imgs)))
train_img_paths = all_val_imgs[:N_TRAIN]

X_batch = []
Y_batch = []
for p in train_img_paths:
    im = cv2.imread(p)
    if im is None:
        continue
    im = cv2.resize(im, (IMG_SIZE, IMG_SIZE))
    X_batch.append(im.astype(np.float32))  # NHWC float32 in 0..255
    lbl = os.path.join(VAL_LABELS_DIR, os.path.splitext(os.path.basename(p))[0] + ".txt")
    Y_batch.append(load_yolo_label_as_art_boxes(lbl, IMG_SIZE, IMG_SIZE))

if len(X_batch) == 0:
    raise RuntimeError("No images loaded for patch generation subset.")

X_batch = np.stack(X_batch, axis=0)                 # NHWC
X_batch = np.transpose(X_batch, (0,3,1,2))          # -> NCHW

print(f"Prepared {len(X_batch)} images for RobustDPatch optimization.")

# ---------- configure + run RobustDPatch ----------
attack = RobustDPatch(
    estimator=est,
    patch_shape=(3, PATCH_SIZE, PATCH_SIZE),
    patch_location=(0, 0),
    learning_rate=LEARNING_RATE,
    max_iter=PATCH_ITERS,
    batch_size=BATCH_SIZE,
    targeted=False,
    verbose=True
)

print("Starting patch optimization...")
patch = attack.generate(x=X_batch, reset_patch=True, random_location=True)

# normalize patch to (3,H,W) numpy float32
if isinstance(patch, np.ndarray):
    p_arr = patch
else:
    p_arr = np.asarray(patch)
if p_arr.ndim == 4 and p_arr.shape[0] == 1:
    p_arr = p_arr[0]
if p_arr.ndim == 2:
    p_arr = np.stack([p_arr, p_arr, p_arr], axis=0)
patch = p_arr.astype(np.float32)
assert patch.ndim == 3 and patch.shape[0] == 3, f"Bad patch shape: {patch.shape}"
print("Patch generated with shape:", patch.shape)




DEVICE=cuda, IMG_SIZE=640
Loading Ultralytics model...
ART estimator created.
Prepared 32 images for RobustDPatch optimization.
Starting patch optimization...


RobustDPatch iteration:   0%|          | 0/400 [00:00<?, ?it/s]

Patch generated with shape: (3, 128, 128)


In [ ]:
# save patch (RGB -> BGR for OpenCV)
os.makedirs(ATTACK_ROOT, exist_ok=True)
patch_img = np.transpose(np.clip(patch, 0, 255).astype(np.uint8), (1, 2, 0))
cv2.imwrite(os.path.join(ATTACK_ROOT, "robust_dpatch.png"), cv2.cvtColor(patch_img, cv2.COLOR_RGB2BGR))
print("Saved patch to:", os.path.join(ATTACK_ROOT, "robust_dpatch.png"))

# ---------- apply patch to validation images and save ----------
ensure_clean_dir(ATTACK_IMG_DIR)
ensure_clean_dir(ATTACK_LBL_DIR)
for p in glob.glob(os.path.join(VAL_LABELS_DIR, "*.txt")):
    shutil.copy2(p, ATTACK_LBL_DIR)

mask = getattr(attack, "mask", None) or getattr(attack, "_mask", None)
if mask is not None:
    mask = np.asarray(mask)
    if mask.ndim == 3 and mask.shape[0] == 1:
        mask = mask[0]
    mask = np.squeeze(mask).astype(bool)

max_x = IMG_SIZE - PATCH_SIZE
max_y = IMG_SIZE - PATCH_SIZE
if max_x < 0 or max_y < 0:
    raise ValueError("PATCH_SIZE must be <= IMG_SIZE")

print("Applying patch to validation images...")
for ip in tqdm(all_val_imgs, desc="ApplyPatch"):
    im = cv2.imread(ip)
    if im is None:
        continue
    im = cv2.resize(im, (IMG_SIZE, IMG_SIZE))
    rgb = cv2.cvtColor(im, cv2.COLOR_BGR2RGB).astype(np.float32)
    x_nchw = np.transpose(rgb, (2, 0, 1))[None, ...].astype(np.float32)

    rand_x = random.randint(0, max_x)
    rand_y = random.randint(0, max_y)
    attack.patch_location = (rand_x, rand_y)

    try:
        if mask is None:
            adv = attack.apply_patch(x=x_nchw, patch_external=patch)
        else:
            adv = attack.apply_patch(x=x_nchw, patch_external=patch, mask=mask)
    except TypeError:
        # fallback signatures if ART version expects positional args
        try:
            if mask is None:
                adv = attack.apply_patch(x_nchw, patch)
            else:
                adv = attack.apply_patch(x_nchw, patch, mask)
        except Exception as ex:
            print("apply_patch failed for", ip, ":", ex)
            continue

    adv_arr = np.asarray(adv)
    if adv_arr.ndim == 4 and adv_arr.shape[0] >= 1:
        adv0 = adv_arr[0]
    elif adv_arr.ndim == 3:
        adv0 = adv_arr
    else:
        print("Unexpected adv shape:", adv_arr.shape, "for", ip)
        continue

    if adv0.dtype != np.uint8:
        adv0 = np.clip(adv0, 0, 255).astype(np.uint8)
    adv_hwc = np.transpose(adv0, (1, 2, 0))
    out_bgr = cv2.cvtColor(adv_hwc, cv2.COLOR_RGB2BGR)
    cv2.imwrite(os.path.join(ATTACK_IMG_DIR, os.path.basename(ip)), out_bgr)

print("Patched images written to:", ATTACK_IMG_DIR)

# ---------- quick sanity check on a single sample ----------
try:
    sample_orig = X_batch[0:1]  # NCHW float32 0..255
    print("\nSanity-check: predictions on original sample")
    print("Original:", est.predict(sample_orig))
    attack.patch_location = (0, 0)
    adv_sample = attack.apply_patch(x=sample_orig, patch_external=patch)
    print("Adversarial:", est.predict(adv_sample))
except Exception as e:
    print("Sanity-check failed:", e)

# ---------- run ultralytics validation on the patched set ----------
print("\nRunning YOLO validation on attacked dataset...")
DATA_YAML = os.path.join(ATTACK_ROOT, "dpatch_attacked_tmp.yaml")
with open(DATA_YAML, "w") as f:
    yaml.safe_dump({
        "path": ATTACK_ROOT,
        "train": "images/val",
        "val": "images/val",
        "names": {i: n for i, n in enumerate(NAMES)},
        "nc": len(NAMES)
    }, f, sort_keys=False)

metrics_attack = yolo.val(
    data=DATA_YAML,
    split="val",
    imgsz=IMG_SIZE,
    conf=CONF_THRES,
    iou=0.5,
    device=DEVICE,
    half=False,
    verbose=False
)

print("\n== RobustDPatch Results on Patched Set ==")
try:
    print(f"mAP@0.50      : {metrics_attack.box.map50:.4f}")
    print(f"mAP@0.50:0.95 : {metrics_attack.box.map:.4f}")
    print(f"Precision     : {metrics_attack.box.mp:.4f}")
    print(f"Recall        : {metrics_attack.box.mr:.4f}")
except Exception:
    print("Could not read metrics fields; raw metrics object:", metrics_attack)

Saved patch to: /content/drive/MyDrive/MN-20-Credit/dota-yolo/DPatch_Attack/robust_dpatch.png
Applying patch to validation images...


ApplyPatch: 100%|██████████| 374/374 [01:20<00:00,  4.67it/s]


Patched images written to: /content/drive/MyDrive/MN-20-Credit/dota-yolo/DPatch_Attack/images/val

Sanity-check: predictions on original sample
Original: [{'boxes': array([[     118.69,      285.06,      176.28,      328.62],
       [     266.79,       147.5,      290.54,      213.15],
       [     426.91,      327.59,      482.56,      359.37],
       [     461.46,      365.73,      500.21,      394.29],
       [     157.16,      387.32,       189.8,      414.21],
       [     104.16,      214.31,      139.64,      243.63],
       [     357.88,      129.07,      380.69,      193.21],
       [     142.34,      418.09,      175.12,      443.41],
       [     113.62,      261.84,      149.29,      289.02],
       [     163.06,      169.29,      182.87,      222.75],
       [     129.18,      349.32,      157.56,      373.44],
       [     327.06,       136.8,      349.65,         203],
       [     213.36,      158.36,      235.55,      222.64],
       [     129.16,      370.17,      160